# Agent from Scratch - using ConverseAPI ToolCalling Feature



This notebook introduces how to build a simple reactive agent using AWS Bedrock Converse SDK. The agent will interact with a conversational model to process and respond to messages.

In [1]:
import boto3, json
import re
from enum import Enum
from datetime import datetime

# Create a session with AWS and initialize a Bedrock client for conversational AI models
session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime')


In [2]:

class Models(Enum):
    # Enum for storing model identifiers for different AI models.
    # Each member represents a specific model hosted by various platforms like Anthropic and Meta.
    
    Sonnet = "anthropic.claude-3-sonnet-20240229-v1:0"  # Identifier for the 'Sonnet' model by Anthropic.
    Haiku = "anthropic.claude-3-haiku-20240307-v1:0"   # Identifier for the 'Haiku' model by Anthropic.
    Llama = "meta.llama3-8b-instruct-v1:0"            # Identifier for the 'Llama' model by Meta.
    Cohere = "cohere.command-r-plus-v1:0"      # Identifier for the 'Cohere' model.

In [3]:
class Agent:
    def calculate(self, expression):
        return str(eval(expression))

    def average_dog_weight(self, name):
        if name in "Scottish Terrier": 
            return("Scottish Terriers average 20 lbs")
        elif name in "Border Collie":
            return("a Border Collies average weight is 37 lbs")
        elif name in "Toy Poodle":
            return("a toy poodles average weight is 7 lbs")
        else:
            print(">>"+name)
            return("An average dog weights 50 lbs")
    

In [4]:
#Define the configuration for our tool...
toolConfig = {'tools': [],
'toolChoice': {
    'auto': {},
    #'any': {},
    #'tool': {
    #    'name': 'get_weather'
    #}
    }
}


In [5]:
# Adding specification for average_dog_weight function
toolConfig['tools'].append({
        'toolSpec': {
            'name': 'average_dog_weight',
            'description': 'Get the average weight of dog based on the breed',
            'inputSchema': {
                'json': {
                    'type': 'object',
                    'properties': {
                        'name': {
                            'type': 'string',
                            'description': 'name of the breed of dog'
                        }
                    },
                    'required': ['name']
                }
            }
        }
    })

In [6]:
# Adding specification for calculate function
toolConfig['tools'].append({
        'toolSpec': {
            'name': 'calculate',
            'description': 'Calculates the mathematical query provided and generates the numerical result',
            'inputSchema': {
                'json': {
                    'type': 'object',
                    'properties': {
                        'expression': {
                            'type': 'string',
                            'description': 'contains the mathematical query'
                        }
                    },
                    'required': ['expression']
                }
            }
        }
    })

In [7]:
#Function for caling the Bedrock Converse API...
def converse_with_tools(model: Models, messages, system='', toolConfig=toolConfig):
    m=model.value
    response = bedrock.converse(
        modelId=m,
        messages=messages,
        inferenceConfig={
                "maxTokens": 2000,  # Maximum number of tokens that the model can generate.
                "temperature": 0.0  # Temperature controls the randomness of the response; 0.0 makes it deterministic.
            },
        toolConfig=toolConfig
    )
    return response

### Add loop to automate actions

In [8]:
# Function for orchestrating the conversation flow with a model that might require additional tools.
def converse_multi(model: Models, prompt, system=''):
    # Initialize a list to keep track of the messages sent and received.
    messages = []

    # Add the initial user prompt to the messages list.
    messages.append({
        "role": "user",
        "content": [{"text": prompt}]
    })

    # Log the initial prompt for debugging purposes.
    print(f"\n- Initial prompt:\n{json.dumps(messages, indent=2)}")

    # Invoke the model the first time using the initial set of messages.
    output = converse_with_tools(model, messages, system)

    # Add the intermediate output to the prompt for further interaction.
    output_message = output['output']['message']
    messages.append(output_message)
    stop_reason = output['stopReason']

    # Continue the conversation as long as the model uses external tools.
    while stop_reason == 'tool_use':
        tool_requests = output['output']['message']['content']
        #print(json.dumps( output['output'], indent=2))
        tresult=[]
        for tool_request in tool_requests:
            if 'toolUse' in tool_request:
                tool = tool_request['toolUse']
                print("Requesting tool: " + tool['name'])
                print("Requesting tool ID: " + tool['toolUseId'])
                
                # Handle specific tools based on the name.
                if tool['name'] == 'average_dog_weight':
                    agent = Agent()
                    breedname = tool['input']['name']
                    dog_weight = agent.average_dog_weight(breedname)
                    tool_result = {
                        "toolUseId": tool['toolUseId'],
                        "content": [{"text": dog_weight}]
                    }

                elif tool['name'] == 'calculate':
                    agent = Agent()
                    expression = tool['input']['expression']
                    calc = agent.calculate(expression)
                    tool_result = {
                        "toolUseId": tool['toolUseId'],
                        "content": [{"text": calc}]
                    }

                else:
                    tool_result = {
                        "toolUseId": tool['toolUseId'],
                        "content": [{"text": "Tool not found"}],
                        "status": 'error'
                    }
                tresult.append({"toolResult": tool_result})
                
        # Construct a new message with the tool result and append it to messages.
        tool_result_message = {
                    "role": "user",
                    "content": tresult
                }
        messages.append(tool_result_message)
        # Send the updated messages list back to the model.
        output = converse_with_tools(model, messages, system)
        output_message = output['output']['message']
        messages.append(output_message)
        stop_reason = output['stopReason']
        print("-------------")
                
    # Return the final message content.
    return output_message["content"][0]['text']


In [9]:
question = """I have 2 dogs, a Border Collie and a Scottish Terrier. What is their combined weight?"""

In [10]:
print("Answer: ", converse_multi(Models.Haiku,question))


- Initial prompt:
[
  {
    "role": "user",
    "content": [
      {
        "text": "I have 2 dogs, a Border Collie and a Scottish Terrier. What is their combined weight?"
      }
    ]
  }
]
Requesting tool: average_dog_weight
Requesting tool ID: tooluse_yjuv3zzhSLOD_lwBFhrtvA
-------------
Requesting tool: average_dog_weight
Requesting tool ID: tooluse_kUEslb5wS-uV89EICRat5Q
-------------
Requesting tool: calculate
Requesting tool ID: tooluse_bd-zwP65Q8W8Higcil9qMg
-------------
Answer:  The combined weight of your Border Collie and Scottish Terrier is 57 lbs.


In [11]:
print("Answer: ", converse_multi(Models.Cohere,question))


- Initial prompt:
[
  {
    "role": "user",
    "content": [
      {
        "text": "I have 2 dogs, a Border Collie and a Scottish Terrier. What is their combined weight?"
      }
    ]
  }
]
Requesting tool: average_dog_weight
Requesting tool ID: tooluse_i_DIAOUrS5Sps7LJbyj8hA
Requesting tool: average_dog_weight
Requesting tool ID: tooluse_AITwWO8cSnuItp33PMATkw
-------------
Answer:  The average weight of a Border Collie is 37 lbs and the average weight of a Scottish Terrier is 20 lbs, so the combined weight of the two dogs is 57 lbs.
